# Dubbo
* https://dubbo.apache.org/en/

> 一款云原生微服务开发框架
> 
> 构建具备内置 RPC、流量管控、安全、可观测能力的应用，支持Kubernetes和VM部署环境。

脚手架: https://start.dubbo.apache.org/bootstrap.html

actions: 
- workbench\Java\JavaEE\example-springcloud\alibaba\dubbo

![](https://dubbo.apache.org/imgs/v3/concepts/architecture-2.png)

基于扩展点的微服务生态

![](https://dubbo.apache.org/imgs/v3/feature/extensibility/arc.png)

# 功能

- 微服务开发
- 服务发现: Dubbo 服务发现扩展了多种注册中心组件支持，如 Nacos、Zookeeper、Consul、Redis、kubernetes 等，可以通过配置切换不同实现，同时还支持鉴权、命名空间隔离等配置.
- 负载均衡
  - Weighted Random LoadBalance: 默认算法, 默认权重相同
  - RoundRobin LoadBalance: Nginx的平滑加权轮询算法, 默认权重相同
  - LeastActive LoadBalance: 最少活跃优先 + 加权随机
  - Shortest-Response LoadBalance: 最短响应优先 + 加权随机
  - ConsistentHash LoadBalance: 一致性哈希, 适用于有状态的请求(确定的入参, 确定的提供者)
  - P2C LoadBalance: 随机选择两个节点, 选择连接数较小的节点
  - Adaptive LoadBalance: P2C算法基础上, 选择负载最小的节点
- 流量管控
  - 条件路由规则
  - 标签路由规则
  - 脚本路由规则
  - 动态配置规则
  - 限流 & 熔断
  - Mesh路由
- 通信协议: Dubbo 框架提供了自定义的高性能 RPC 通信协议：基于 HTTP/2 的 Triple 协议 和 基于 TCP 的 Dubbo2 协议。除此之外，Dubbo 框架支持任意第三方通信协议，如官方支持的 gRPC、Thrift、REST、JsonRPC、Hessian2 等，更多协议可以通过自定义扩展实现。
- 扩展适配
  - 协议通信层
  - 流量管控层
  - 服务治理层
- 观测服务
- 认证鉴权
- 服务网格
- 微服务生态
  - 服务发现
  - 动态配置
  - 元数据管理
  - RPC协议
  - 可视化观测平台
  - 全链路追踪
  - 限流降级
  - 分布式事务
  - 网关
  - 服务网格
- 高级功能
  - 控制服务调用行为
  - 诊断与调优


# 2.x 框架设计
* ref: https://dubbo.apache.org/zh-cn/docsv2.7/dev/design/

<img src="https://dubbo.apache.org/imgs/dev/dubbo-framework.jpg" width="800">

- **config 配置层**: 对外配置接口, 以 ServiceConfig, ReferenceConfig 为中心, 可以直接初始化配置类, 也可以通过 spring 解析配置生成配置类
- **proxy 服务代理层**: 服务接口透明代理, 生成服务的客户端 Stub 和服务器端 Skeleton, 以 ServiceProxy 为中心, 扩展接口为 ProxyFactory
- **registry 注册中心层**: 封装服务地址的注册与发现, 以服务 URL 为中心, 扩展接口为 RegistryFactory, Registry, RegistryService
- **cluster 路由层**: 封装多个提供者的路由及负载均衡, 并桥接注册中心, 以 Invoker 为中心, 扩展接口为 Cluster, Directory, Router, LoadBalance
- **monitor 监控层**: RPC 调用次数和调用时间监控, 以 Statistics 为中心, 扩展接口为 MonitorFactory, Monitor, MonitorService
- **protocol 远程调用层**: 封装 RPC 调用, 以 Invocation, Result 为中心, 扩展接口为 Protocol, Invoker, Exporter
- **exchange 信息交换层**: 封装请求响应模式, 同步转异步, 以 Request, Response 为中心, 扩展接口为 Exchanger, ExchangeChannel, ExchangeClient, ExchangeServer
- **transport 网络传输层**: 抽象 mina 和 netty 为统一接口, 以 Message 为中心, 扩展接口为 Channel, Transporter, Client, Server, Codec
- **serialize 数据序列化层**: 可复用的一些工具, 扩展接口为 Serialization, ObjectInput, ObjectOutput, ThreadPool


- 超时内部是怎么实现的?
- 如果调用方已经超时产生异常, 提供者执行完毕后还会向调用方返回值吗?

RPC框架的超时实现原理（Dubbo实现原理）
https://blog.csdn.net/qq_40126996/article/details/106397934

dubbo支持非常细粒度的超时设置，<br/>
包括：方法级别、接口级别和全局。如果各个级别同时配置了，<br/>
优先级为：消费端方法级 > 服务端方法级 > 消费端接口级 > 服务端接口级 > 消费端全局 > 服务端全局。<br/>

*服务端*如果超时了那么也只是通过logger.warn打印一个warn日志。因此服务端的超时设置并不会影响实际的调用过程，就算超时也会执行完所有的处理逻辑。<br/>
FailoverClusterInvoker 是集群容错的缺省模式，当调用失败后会切换成调用其他服务器。再看下doInvoke方法，当调用失败时，会先判断是否是业务异常，如果是则终止重试，否则会一直重试直到达到重试次数。<br/>
在该invoke方法中如果超时没有获得返回结果，那么会抛出TimeoutException异常。因此消费端的超时逻辑同时受到超时时间和超时次数两个参数的控制，像网络异常、响应超时等都会一直重试，直到达到重试次数。

Dubbo（八）：timeout超时机制解析<br/>
https://www.cnblogs.com/yougewe/p/16460724.html<br/>
dubbo的超时机制是通过*异步线程future*的方式实现的，其中，同步调用的超时设置，底层也是用异步实现。<br/>
总体来说就是，在提交服务端的查询请求时，会开启*定时任务*，检查超时。如果定时任务到期，还未收到结果则会触发超时通知。如果客户端还未成功发送数据，则认为是客户端自己超时。如果已经将数据发送出去，则认为是服务端超时。这相当于是一个看门狗的形式处理了，就是说，不管服务端和客户端本身如何，总能被这东西给发现，所以这种超时控制是精确的。<br/>
  异步执行时，使用的就是 `ThreadPoolExecutor`, 直接进行execute, 即提交到线程池立即执行。即都是统一用共享线程池进行处理，这样做的好处是，不需要等待客户端调用结果，而是主动将结果放置到future的result位置，只需等待处理即可。<br/>
server端仅在特殊情况下才会处理超时。它是在 `TimeoutFilter` 做的简单处理，仅将结果清空，然后正常返回了。

